In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from itertools import combinations #will told me about this
%matplotlib inline

In [2]:
#create file
d = pd.read_csv("dummy_data.csv")
df = d[["time_spent", "age", "gender", "interests", "location", "demographics", "profession", "income", "indebt", "isHomeOwner", "Owns_Car"]]

In [3]:
#clean data
df["gender"] = preprocessing.LabelEncoder().fit(["male", "female", "non-binary"]).transform(df["gender"])
df["interests"] = preprocessing.LabelEncoder().fit(["Sports", "Travel", "Lifestlye"]).transform(df["interests"])
df["location"] = preprocessing.LabelEncoder().fit(["United States", "United Kingdom", "Australia"]).transform(df["location"])
df["demographics"] = preprocessing.LabelEncoder().fit(["Urban", "Sub_Urban", "Rural"]).transform(df["demographics"])
df["profession"] = preprocessing.LabelEncoder().fit(["Student", "Software Engineer", "Marketer Manager"]).transform(df["profession"]) #these throw harmless warnings.
df["indebt"] = preprocessing.LabelEncoder().fit(["False", "True"]).transform(df["indebt"])
df["isHomeOwner"] = preprocessing.LabelEncoder().fit(["False", "True"]).transform(df["isHomeOwner"])
df["Owns_Car"] = preprocessing.LabelEncoder().fit(["False", "True"]).transform(df["Owns_Car"])
X = df[["time_spent", "age", "gender", "interests", "location", "demographics", "profession", "income", "indebt", "isHomeOwner", "Owns_Car"]].values.astype(float) 

/var/folders/np/wt2f_g3s42g_nygnmnbpcswr0000gn/T/ipykernel_13890/2201030753.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["gender"] = preprocessing.LabelEncoder().fit(["male", "female", "non-binary"]).transform(df["gender"])
/var/folders/np/wt2f_g3s42g_nygnmnbpcswr0000gn/T/ipykernel_13890/2201030753.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["interests"] = preprocessing.LabelEncoder().fit(["Sports", "Travel", "Lifestlye"]).transform(df["interests"])
/var/folders/np/wt2f_g3s42g_nygnmnbpc

In [4]:
#make variable arrays
features = ["age", "gender", "interests", "demographics", "profession", "income", "indebt", "isHomeOwner", "Owns_Car"]
ind = df[features]
deps = df[["time_spent"]]

In [5]:
#normalize data
ind = pd.DataFrame(preprocessing.StandardScaler().fit(ind).transform(ind))
ind.columns = features


In [6]:
#inspect data
ind.head()

,age,gender,interests,demographics,profession,income,indebt,isHomeOwner,Owns_Car
0,1.112882,-0.001228,0.015896,1.244883,0.056541,1.609380,1.006018,-1.016130,-1.081294
1,0.371652,-1.229356,1.238658,1.244883,1.285703,-1.505106,1.006018,0.984126,0.924818
2,-0.666069,-0.001228,0.015896,0.018397,-1.172620,-0.594094,-0.994018,-1.016130,-1.081294
3,1.409373,1.226900,1.238658,1.244883,1.285703,-0.850422,-0.994018,0.984126,-1.081294
4,-1.184929,-0.001228,-1.206866,1.244883,0.056541,-0.151776,-0.994018,0.984126,0.924818


In [7]:
#create a list of all possible combinations of features
feature_combs = []
for length in range(1, len(features) + 1):
    feature_combs.extend(list(combinations(features, length)))

In [8]:
len(feature_combs)

511

In [9]:
acc_list = []
for comb in feature_combs:
    comb_ind = ind[list(comb)]
    train_d, test_d, train_i, test_i = train_test_split(deps, comb_ind, test_size=0.2, random_state=4) #split data
    for k in range(20, 200): #this loop finds the the accuracy for 100<k<300
        m = KNeighborsClassifier(n_neighbors = k, n_jobs = -1).fit(train_i, np.ravel(train_d)) #fits a model for k = k
        yhat = m.predict(test_i)  #generates predictions for model m on test set
        acc = metrics.accuracy_score(test_d, yhat) #gets accuracy score on test set
        acc_list.append([acc, k])

In [10]:
max(acc_list)

[0.23, 55]

In [11]:
# this is 23% accuracy--better than random (11.1%), but clearly not very good. Let's try LR next.